In [ ]:
columns_name=["mpg",
"cylinders",
"displacement",
"horsepower",
"weight",
"model year",
"origin",
"car name"
]

In [ ]:
data.horsepower

,horsepower
0,130.0
1,165.0
2,150.0
3,150.0
4,140.0
...,...
393,86.0
394,52.0
395,84.0
396,79.0


In [ ]:
import tensorflow as tf


In [ ]:
import pandas as pd
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from sklearn.model_selection import train_test_split

# Load the data
data = pd.read_csv("/content/sample_data/auto-mpg (1).csv")

# Drop the 'car name' column
data_set = data.drop("car name", axis=1)

# Split into features (x) and target (y)
x_train = data_set.drop("mpg", axis=1)
y_train = data_set["mpg"]

# Split into training, validation, and testing sets
x_train, x_test, y_train, y_test = train_test_split(x_train, y_train, test_size=0.2, random_state=42)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.1, random_state=42)

# --- Data Cleaning for 'horsepower' in ALL datasets ---
for df in [x_train, x_val, x_test]:  # Include x_test if you will use it later
    # Replace "?" with None
    df['horsepower'] = df['horsepower'].replace("?", None)
    # Convert to numeric, coercing errors to NaN
    df['horsepower'] = pd.to_numeric(df['horsepower'], errors='coerce')
    # Fill NaN values with the median of 'horsepower'
    median_hp = df['horsepower'].median()
    df['horsepower'] = df['horsepower'].fillna(median_hp)
    # Convert to float
    df['horsepower'] = df['horsepower'].astype(float)

# Create and fit the column transformer
ct = make_column_transformer(
    (StandardScaler(), ["cylinders", "displacement", "horsepower", "weight", "model year", "origin"]),
    remainder='drop'
)
ct.fit(x_train)  # Fit on the training data only


# --- Model Creation and Training ---
model = tf.keras.Sequential([
    tf.keras.layers.Dense(100, activation="relu"),
    tf.keras.layers.Dense(100, activation="relu"),
    tf.keras.layers.Dense(10, activation="linear"),
])
model.compile(loss="mae", optimizer=tf.keras.optimizers.Adam(), metrics=["mae"])
lr_scheduler = tf.keras.callbacks.LearningRateScheduler(lambda epochs: 1e-3 * 10**(epochs/20))
find_lr_history = model.fit(x_train, y_train, epochs=100, validation_data=(x_val, y_val), callbacks=[lr_scheduler])

Epoch 1/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 30ms/step - loss: 86.5830 - mae: 86.5830 - val_loss: 25.7836 - val_mae: 25.7836 - learning_rate: 0.0010
Epoch 2/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 27.0627 - mae: 27.0627 - val_loss: 20.1601 - val_mae: 20.1601 - learning_rate: 0.0011
Epoch 3/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 18.3654 - mae: 18.3654 - val_loss: 15.6264 - val_mae: 15.6264 - learning_rate: 0.0013
Epoch 4/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 14.5535 - mae: 14.5535 - val_loss: 14.0871 - val_mae: 14.0871 - learning_rate: 0.0014
Epoch 5/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 12.9958 - mae: 12.9958 - val_loss: 10.7605 - val_mae: 10.7605 - learning_rate: 0.0016
Epoch 6/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 11.2362 - mae: 11.2362 - val_loss: 11.8361 - val_mae: 11.8361 - learning_rate: 0.0018
Epoch 7/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 11.1311 - mae: 11.1311 - val_loss: 11.0192 - val_mae: 11.0192 - learning_rate: 0.002

In [ ]:
model.evaluate(x_test, y_test)

3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - loss: 7.0687 - mae: 7.0687 


[6.885772705078125, 6.885772705078125]

In [ ]:
y_pred=model.predict(x_test)

3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step


In [ ]:
y_pred

array([[26.141804, 26.322361, 27.095356, 23.221958, 30.726015, 26.511293,
        29.51046 , 25.815868, 21.831873, 26.171967],
       [26.141804, 26.322361, 27.095356, 23.221958, 30.726015, 26.511293,
        29.51046 , 25.815868, 21.831873, 26.171967],
       [26.141804, 26.322361, 27.095356, 23.221958, 30.726015, 26.511293,
        29.51046 , 25.815868, 21.831873, 26.171967],
       [26.141804, 26.322361, 27.095356, 23.221958, 30.726015, 26.511293,
        29.51046 , 25.815868, 21.831873, 26.171967],
       [26.141804, 26.322361, 27.095356, 23.221958, 30.726015, 26.511293,
        29.51046 , 25.815868, 21.831873, 26.171967],
       [26.141804, 26.322361, 27.095356, 23.221958, 30.726015, 26.511293,
        29.51046 , 25.815868, 21.831873, 26.171967],
       [26.141804, 26.322361, 27.095356, 23.221958, 30.726015, 26.511293,
        29.51046 , 25.815868, 21.831873, 26.171967],
       [26.141804, 26.322361, 27.095356, 23.221958, 30.726015, 26.511293,
        29.51046 , 25.815868, 21.8318